In [ ]:
https://kartaslov.ru/синонимы-к-слову/слово

In [207]:
import os
import requests
from bs4 import BeautifulSoup
from pymorphy2 import MorphAnalyzer
from rnnmorph.predictor import RNNMorphPredictor

predictor = RNNMorphPredictor(language="ru")
morph = MorphAnalyzer()

In [256]:
[(x,z,y) for x,y,z in [[1,2,3],[4,5,6]]]

[(1, 3, 2), (4, 6, 5)]

In [301]:
from time import sleep

In [396]:
def find_synonym(normal_form, gender=None):
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    if os.path.exists('synonyms.txt'):
        with open('synonyms.txt') as synonyms_cache:
            synonyms_cache = [x.strip().split() for x in synonyms_cache]
            synonyms_cache = {(form, gen):syn for form, gen, syn in synonyms_cache}
        if (normal_form, str(gender)) in synonyms_cache:
            #print('yes')
            return synonyms_cache[(normal_form, str(gender))]
    try:
        content = requests.get(f'https://kartaslov.ru/синонимы-к-слову/{normal_form}', headers=headers).content
    except ConnectionError:
        print('Sleeping')
        sleep(5)
        try:
            content = requests.get(f'https://kartaslov.ru/синонимы-к-слову/{normal_form}', headers=headers).content
        except ConnectionError:
            print('no')
            return None
    soup = BeautifulSoup(content)
    #print(normal_form)
    synonyms_page = soup.findAll('ul')
    if len(synonyms_page) > 1:
        synonyms = sum([x.text.strip().split(', ') for x in synonyms_page[1].findAll('li')], [])
        synonyms = [x for x in synonyms if ' ' not in x and '\xa0' not in x]
    else:
        return None
    
    # ADD CHECK WITH W2V
    #print(synonyms)
    if not synonyms:
        return None
    if gender:
        try:
            synonym = list(filter(lambda x: gender in x.tag, predictor.predict(synonyms)))[0].word
        except IndexError:
            synonym = None
    else:
        synonym = synonyms[0]
    #print(synonym)
    if synonym is not None:
        with open('synonyms.txt', 'a') as synonyms_cache:
            synonyms_cache.write(f'{normal_form} {gender} {synonym}'+'\n')
    return synonym

In [397]:
def replace_nouns(source_sentence, num=1):
    target_sentence = source_sentence.copy()

    candidate_forms = list(filter(lambda x: x.pos == 'NOUN', predictor.predict(source_sentence)))[:num]
    for form in candidate_forms:
        case_num = re.search('case=(.+)\|.+number=(.+)', form.tag.lower())
        gender = re.search('Gender=(.+)\|', form.tag)
        if gender:
            gender = gender.group(1)
        else:
            continue
        tags = {transforms[case_num.group(1)], case_num.group(2)}
        
        synonym = find_synonym(form.normal_form, gender)
        if synonym is None:
            continue
        inflected_word = list(filter(lambda x : x.tag.POS in ['NOUN'], morph.parse(synonym)))
        if inflected_word:
            #print(inflected_word[0], tags, inflected_word[0].inflect(tags))
            inflected_word = inflected_word[0].inflect(tags)
            if inflected_word:
                inflected_word = inflected_word.word
            else:
                continue
        else:
            continue
        target_sentence[target_sentence.index(form.word)] = inflected_word
    return target_sentence

In [364]:
form = '<normal_form=сделанный; word=сделано; pos=ADJ; tag=Degree=Pos|Gender=Neut|Number=Sing|Variant=Short; score=0.9999>'
case = re.search('case=(.{3})\|.+', form.lower())
num = re.search('number=(.{4})\|.+', form.lower())
case.group(1), 
num.group(1)

'sing'

In [393]:
replace_nouns(['травы'])

yes


['травы']

In [398]:
def replace_adjectives(source_sentence, num=1):
    target_sentence = source_sentence.copy()

    candidate_forms = list(filter(lambda x: x.pos == 'ADJ', predictor.predict(source_sentence)))[:num]
    for form in candidate_forms:
        case = re.search('case=(.{3})\|.+', form.tag.lower())
        num = re.search('number=(.{4})', form.tag.lower())
        #print(form)
        if not num:
            continue
        if num.group(1) == 'plur':
            gender = None
            tags = {transforms[case.group(1)], num.group(1)}
        else:
            gender = re.search('gender=(.{3,4})\|.+', form.tag.lower()).group(1)
            if not case:
                #print(12, num.group(1), case_num)
                tags = {num.group(1), transforms[gender]}
            else:
                tags = {transforms[case.group(1)], num.group(1), transforms[gender]}
        
        #print(tags)
        synonym = find_synonym(form.normal_form)
        if synonym is None:
            continue
        inflected_word = list(filter(lambda x : x.tag.POS in ['ADJF'], morph.parse(synonym)))
        if inflected_word:
            inflected_word = inflected_word[0].inflect(tags).word
        else:
            continue
        target_sentence[target_sentence.index(form.word)] = inflected_word
    return target_sentence

In [220]:
def replace_adverbs(source_sentence, num=1):
    target_sentence = source_sentence.copy()

    candidate_forms = list(filter(lambda x: x.pos == 'ADV', predictor.predict(source_sentence)))[:num]
    for form in candidate_forms:
        synonym = find_synonym(form.normal_form)
        target_sentence[target_sentence.index(form.word)] = synonym
    return target_sentence

In [399]:
def modify_sentences(src, tgt):
    modified_src = []
    modified_tgt = []
    for num, (src_sentence, sentence) in enumerate(zip(src, tgt)):
        #print(sentence)
        modified_sentence = replace_nouns(sentence)
        if modified_sentence == sentence:
            modified_sentence = replace_adverbs(sentence)
            if modified_sentence == sentence:
                modified_sentence = replace_adjectives(sentence)
                if modified_sentence == sentence:
                    continue
        modified_src.append(src_sentence)
        modified_tgt.append(modified_sentence)
        if num % 50 == 0:
            print(f'{num}/{len(src)}')
    return modified_src, modified_tgt

In [400]:
language = 'datasets/evenki'
#os.mkdir(f'{language}_aug')
with open(f'{language}/src-train14r.txt') as srctrain, open(f'{language}/tgt-train14r.txt') as tgttrain, open(f'{language}_aug/tgt-train.txt', 'w') as aug_srctrain, open(f'{language}_aug/tgt-train.txt', 'w') as aug_tgttrain:
        srctrain = [x.strip().split() for x in srctrain]
        tgttrain = [x.strip().split() for x in tgttrain]
        aug_src, aug_tgt = modify_sentences(srctrain, tgttrain)
        for src_line, tgt_line in zip(srctrain, tgttrain):
            aug_srctrain.write(' '.join(src_line)+'\n')
            aug_tgttrain.write(' '.join(tgt_line)+'\n')
        for src_line, tgt_line in zip(aug_src, aug_tgt):
            aug_srctrain.write(' '.join(src_line)+'\n')
            aug_tgttrain.write(' '.join(tgt_line)+'\n')
            

yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes


yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes


NameError: name 'tgt_line' is not defined

In [199]:
src = evenki[:20]
tgt = evenki[:20]

In [205]:
len(mod_src), len(src)

(16, 20)

In [206]:
[print(x, y) for x, y in zip(mod_src,mod_tgt)]

['так', 'живем', 'сейчас'] ['примерно', 'живем', 'сейчас']
['костры', 'разжег', 'отец'] ['крепи', 'разжег', 'отец']
['потом', 'большой'] ['следом', 'большой']
['говорит', 'если', 'олени', 'потеряются'] ['говорит', 'если', 'лоси', 'потеряются']
['у', 'тебя', 'опыт', 'большой'] ['у', 'тебя', 'навык', 'большой']
['я', 'в', 'берлогу', 'зашла'] ['я', 'в', 'нору', 'зашла']
['там', 'один', 'или', 'два', 'года', 'все', 'время', 'один'] ['там', 'один', 'или', 'два', 'годка', 'все', 'время', 'один']
['оттуда', 'мы', 'в', 'стадо', 'аргишили'] ['оттуда', 'мы', 'в', 'поголовье', 'аргишили']
['дедушка', 'это', 'кто', 'ходит', 'ломая', 'коряги', 'деревья'] ['дед', 'это', 'кто', 'ходит', 'ломая', 'коряги', 'деревья']
['они', 'ранее', 'комары'] ['они', 'ранее', 'комарики']
['на', 'холодето', 'железо', 'далеко', 'идет', 'т', 'е', 'звучит', 'если', 'закрываешь', 'ружье'] ['на', 'холодето', 'железка', 'далеко', 'идет', 'т', 'е', 'звучит', 'если', 'закрываешь', 'ружье']
['ее', 'муж', 'работает', 'врачом', 

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [169]:
morph.parse('красиво')

[Parse(word='красиво', tag=OpencorporaTag('ADVB'), normal_form='красиво', score=0.75, methods_stack=((DictionaryAnalyzer(), 'красиво', 3, 0),)),
 Parse(word='красиво', tag=OpencorporaTag('ADJS,Qual neut,sing'), normal_form='красивый', score=0.25, methods_stack=((DictionaryAnalyzer(), 'красиво', 1878, 56),))]

In [167]:
predictor.predict('я красиво иду'.split())

[<normal_form=я; word=я; pos=PRON; tag=Case=Nom|Number=Sing|Person=1; score=1.0000>,
 <normal_form=красиво; word=красиво; pos=ADV; tag=Degree=Pos; score=0.9999>,
 <normal_form=идти; word=иду; pos=VERB; tag=Mood=Ind|Number=Sing|Person=1|Tense=Notpast|VerbForm=Fin|Voice=Act; score=1.0000>]

In [125]:
import pandas as pd

In [126]:
df = pd.read_csv('datasets/evenki.csv')

In [133]:
with open('datasets/evenki/tgt-train14r.txt') as evenki:
    evenki = [x.strip().split() for x in evenki.readlines()]
evenki[:10]

[['так', 'живем', 'сейчас'],
 ['костры', 'разжег', 'отец'],
 ['потом', 'большой'],
 ['говорит', 'если', 'олени', 'потеряются'],
 ['у', 'тебя', 'опыт', 'большой'],
 ['я', 'в', 'берлогу', 'зашла'],
 ['там', 'один', 'или', 'два', 'года', 'все', 'время', 'один'],
 ['оттуда', 'мы', 'в', 'стадо', 'аргишили'],
 ['дедушка', 'это', 'кто', 'ходит', 'ломая', 'коряги', 'деревья'],
 ['сказала', 'это', 'самое']]

In [135]:
for sent in evenki[:10]:
    trans_sent = replace_nouns(sent)
    print(trans_sent, sent)

['так', 'живем', 'сейчас'] ['так', 'живем', 'сейчас']
['крепи', 'разжег', 'отец'] ['костры', 'разжег', 'отец']
['потом', 'большой'] ['потом', 'большой']
['говорит', 'если', 'лоси', 'потеряются'] ['говорит', 'если', 'олени', 'потеряются']
['у', 'тебя', 'навык', 'большой'] ['у', 'тебя', 'опыт', 'большой']
['я', 'в', 'нору', 'зашла'] ['я', 'в', 'берлогу', 'зашла']
['там', 'один', 'или', 'два', 'годка', 'все', 'время', 'один'] ['там', 'один', 'или', 'два', 'года', 'все', 'время', 'один']
['оттуда', 'мы', 'в', 'поголовье', 'аргишили'] ['оттуда', 'мы', 'в', 'стадо', 'аргишили']
['дед', 'это', 'кто', 'ходит', 'ломая', 'коряги', 'деревья'] ['дедушка', 'это', 'кто', 'ходит', 'ломая', 'коряги', 'деревья']
['сказала', 'это', 'самое'] ['сказала', 'это', 'самое']


In [165]:
for sent in evenki[:10]:
    trans_sent = replace_adjectives(sent)
    print(trans_sent, sent)

['так', 'живем', 'сейчас'] ['так', 'живем', 'сейчас']
['костры', 'разжег', 'отец'] ['костры', 'разжег', 'отец']
nom sing
['потом', 'больший'] ['потом', 'большой']
['говорит', 'если', 'олени', 'потеряются'] ['говорит', 'если', 'олени', 'потеряются']
nom sing
['у', 'тебя', 'опыт', 'больший'] ['у', 'тебя', 'опыт', 'большой']
['я', 'в', 'берлогу', 'зашла'] ['я', 'в', 'берлогу', 'зашла']
nom sing
['там', 'какой-то', 'или', 'два', 'года', 'все', 'время', 'один'] ['там', 'один', 'или', 'два', 'года', 'все', 'время', 'один']
['оттуда', 'мы', 'в', 'стадо', 'аргишили'] ['оттуда', 'мы', 'в', 'стадо', 'аргишили']
['дедушка', 'это', 'кто', 'ходит', 'ломая', 'коряги', 'деревья'] ['дедушка', 'это', 'кто', 'ходит', 'ломая', 'коряги', 'деревья']
acc sing
['сказала', 'это', 'само'] ['сказала', 'это', 'самое']


In [178]:
for sent in evenki[:10]:
    trans_sent = replace_adverbs(sent)
    print(trans_sent, sent, trans_sent==sent)

['примерно', 'живем', 'сейчас'] ['так', 'живем', 'сейчас'] False
['костры', 'разжег', 'отец'] ['костры', 'разжег', 'отец'] True
['следом', 'большой'] ['потом', 'большой'] False
['говорит', 'если', 'олени', 'потеряются'] ['говорит', 'если', 'олени', 'потеряются'] True
['у', 'тебя', 'опыт', 'большой'] ['у', 'тебя', 'опыт', 'большой'] True
['я', 'в', 'берлогу', 'зашла'] ['я', 'в', 'берлогу', 'зашла'] True
['а там', 'один', 'или', 'два', 'года', 'все', 'время', 'один'] ['там', 'один', 'или', 'два', 'года', 'все', 'время', 'один'] False
['оттудова', 'мы', 'в', 'стадо', 'аргишили'] ['оттуда', 'мы', 'в', 'стадо', 'аргишили'] False
['дедушка', 'это', 'кто', 'ходит', 'ломая', 'коряги', 'деревья'] ['дедушка', 'это', 'кто', 'ходит', 'ломая', 'коряги', 'деревья'] True
['сказала', 'это', 'самое'] ['сказала', 'это', 'самое'] True


In [111]:
sentence = 'я люблю енотов'.split()
sentence[sentence.index('енотов')] = 'котов'
sentence

['я', 'люблю', 'котов']

In [101]:
forms = filter(lambda x: x.pos == 'NOUN', predictor.predict('я люблю есть помидоры и свеклу'.split()))
list(forms)

[<normal_form=помидор; word=помидоры; pos=NOUN; tag=Case=Acc|Gender=Masc|Number=Plur; score=0.9996>,
 <normal_form=свёкла; word=свеклу; pos=NOUN; tag=Case=Acc|Gender=Fem|Number=Sing; score=0.9998>]

In [27]:
word = 'мама'


In [99]:
forms = predictor.predict('печь печи печи печь печью рамы раме о раме рамой'.split())
f = forms[2].tag
f, forms[0].pos

('Case=Gen|Gender=Fem|Number=Sing', 'VERB')

In [163]:
transforms = {'nom': 'nomn',
              'gen': 'gent',
              'dat': 'datv',
              'acc': 'accs',
              'ins': 'ablt',
              'loc': 'loct',
              'masc': 'masc',
              'fem': 'femn',
              'neut': 'neut'}

In [95]:
import re
res = re.search('case=(.+)\|.+number=(.+)', f.lower())
ans = {transforms[res.group(1)], res.group(2)}

inflected_word = list(filter(lambda x : x.tag.POS in ['NOUN'], morph.parse(synonym_word)))[0].inflect(ans).word

'помаду'